In [51]:
#We install all of the packages necessary
#Sodapy is for Socrata
!pip3 install sodapy
!pip3 install -U pymongo
!pip install -U flask
!pip3 install tomlkit
!pip install -U pyspark

In [52]:
#We import the basic packages to read dataframe, API, Json files, and MongoDB
import pandas as pd
from sodapy import Socrata
import json 
from json import loads
from pymongo import MongoClient
from flask import Flask, request, render_template

In [53]:
#We start and run MongoDB, we call our database TermProject
client1 = MongoClient('localhost',27017)
dbb = client1.TermProject
collection = dbb.test_collection
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'TermProject'), 'test_collection')

In [54]:
#This extracts the host server, which is the NYS Governmental Website
client = Socrata("data.ny.gov", None)

Above, we install and import our packages, and run MongoDB. This is the start of our project.

In [55]:
start_date = '2023-03-01'
end_date = '2023-03-31'
rider = client.get("wujg-7c2s", where=f"transit_timestamp >= '{start_date}' AND transit_timestamp <= '{end_date}'",limit=2500000)

Above, we request to retrieve data from NY's dataset identified by the API ID "wujg-7c2s" within March 2023. The request is filtered to include only records with a "transit_timestamp". The limit parameter is set to 2,500,000 rows. This is how we acquire all 4 of our datasets

In [56]:
#Convert into dataframe using PD
ridership = pd.DataFrame.from_records(rider)
ridership[['Date', 'Time']] = ridership['transit_timestamp'].str.split('T', expand=True)

#We split the data and time using str.split, then we extract the day and print a new column that indicates weekday or weekend.
from datetime import datetime
def day_type(date_str):
    day = int(date_str.split('-')[2])
    if (day + 4) % 7 < 5:
        return 1
    else:
        return 2

ridership['day_type'] = ridership['Date'].apply(lambda x: day_type(x))

In [57]:
ridership = ridership[~ridership['station_complex_id'].str.contains('TRAM')]
ridership.station_complex_id.unique()

array(['28', '280', '282', '283', '284', '286', '287', '288', '289',
       '290', '190', '291', '292', '293', '294', '295', '296', '297',
       '298', '299', '3', '30', '300', '301', '303', '304', '305', '306',
       '307', '308', '309', '31', '310', '311', '312', '313', '314',
       '318', '316', '319', '32', '320', '321', '323', '324', '325',
       '326', '327', '328', '329', '33', '333', '334', '153', '336',
       '337', '339', '34', '340', '341', '343', '344', '345', '346',
       '347', '348', '349', '35', '350', '351', '352', '353', '354',
       '355', '356', '357', '358', '359', '36', '360', '361', '362',
       '363', '364', '365', '366', '367', '368', '369', '37', '370',
       '371', '372', '373', '374', '375', '376', '377', '378', '379',
       '38', '380', '381', '382', '383', '384', '385', '386', '387',
       '388', '39', '391', '392', '393', '394', '395', '396', '397',
       '398', '399', '4', '403', '404', '405', '407', '409', '41', '413',
       '414', '416', '

In [58]:
ridership[['latitude','longitude']] = ridership[['latitude','longitude']].astype(float)
ridership.dtypes

transit_timestamp       object
transit_mode            object
station_complex_id      object
station_complex         object
borough                 object
payment_method          object
fare_class_category     object
ridership               object
transfers               object
latitude               float64
longitude              float64
georeference            object
Date                    object
Time                    object
day_type                 int64
dtype: object

These 3 cells above are pre-processing the dataset. We separate time_stampe into Date and Time. We remove any Station_Complex_ID that has 'TRAM' to remove the Roosevelt Island Tram. We convert coordinates into float and day_type into integer. We set the Day_type to 1 (weekday) and 2(weekend) as the rest of the datasets used only use day types.

In [59]:
rider_json = ridership.to_json(orient='records')
rider_json = json.loads(rider_json)
collection = dbb.rider_json
collection.insert_many(rider_json)

print(f"{len(rider_json)} documents inserted into MongoDB successfully.")

2100300 documents inserted into MongoDB successfully.


In [60]:
#client1.close()

#### Q1: Which Stations need to be be fully express?

*An Express Station contains local and express trains of the same color group* 

These next several queries will show the process of finding express stations by line.

In [61]:
# Define the aggregation pipeline to calculate the number of swipes by day type
p0 = [
    # Match documents where the borough is not Staten Island (Subways Do not go there)
    {"$match": {"borough": {"$ne": "Staten"}}},
    # Group documents by day type and calculate the sum of swipes for each group
    {"$group": {"_id": "$day_type", "ridership": {"$sum": 1}}},
    # Project the _id field as day_type and include the ridership count, exclude the _id field
    {"$project": {"day_type": "$_id", "ridership": 1, "_id": 0}},
    # Sort the results by ridership count in descending order
    {"$sort": {"ridership": -1}}
]

# Execute the aggregation pipeline
r0 = collection.aggregate(p0)

# Print the results
print("Number of Swipes by Day_Type")
for result in r0:
    print(result)
print("Day_Type 1 is weekday, 2 is weekend")

Number of Swipes by Day_Type
{'ridership': 6847885, 'day_type': 1}
{'ridership': 3653615, 'day_type': 2}
Day_Type 1 is weekday, 2 is weekend


In [62]:
# Pipeline to calculate the number of swipes by borough
p1 = [
    # Group documents by borough and calculate the sum of swipes for each group
    {"$group": {"_id": "$borough", "ridership": {"$sum": 1}}},
    # Project the _id field as borough and include the ridership count
    {"$project": {"borough": "$_id", "ridership": 1, "_id": 0}},
    # Sort the ridership count in descending order
    {"$sort": {"ridership": -1}}
]

r1 = collection.aggregate(p1)

# Print the results
print("Number of Swipes by Borough")
for result in r1:
    print(result)

Number of Swipes by Borough
{'ridership': 3764970, 'borough': 'Brooklyn'}
{'ridership': 3196640, 'borough': 'Manhattan'}
{'ridership': 1850215, 'borough': 'Queens'}
{'ridership': 1647750, 'borough': 'Bronx'}
{'ridership': 41925, 'borough': 'Staten Island'}


In [63]:
#Calculate the number of swipes by card type
p2 = [
    # Group documents by fare class category and calculate the sum of swipes for each group
    {"$group": {"_id": "$fare_class_category", "ridership": {"$sum": 1}}},
    # Project the _id field as fare_class_category and include the ridership count
    {"$project": {"fare_class_category": "$_id", "ridership": 1, "_id": 0}},
    # Sort ridership count in descending order
    {"$sort": {"ridership": -1}}
]

r2 = collection.aggregate(p2)

# Print the results
print("Number of Swipes by Card Type")
for result in r2:
    print(result)

Number of Swipes by Card Type
{'ridership': 1452020, 'fare_class_category': 'OMNY - Full Fare'}
{'ridership': 1438940, 'fare_class_category': 'Metrocard - Full Fare'}
{'ridership': 1412240, 'fare_class_category': 'Metrocard - Unlimited 7-Day'}
{'ridership': 1364470, 'fare_class_category': 'Metrocard - Unlimited 30-Day'}
{'ridership': 1325850, 'fare_class_category': 'Metrocard - Other'}
{'ridership': 1255805, 'fare_class_category': 'Metrocard - Fair Fare'}
{'ridership': 1154690, 'fare_class_category': 'Metrocard - Seniors & Disability'}
{'ridership': 779390, 'fare_class_category': 'Metrocard - Students'}
{'ridership': 292455, 'fare_class_category': 'OMNY - Seniors & Disability'}
{'ridership': 25640, 'fare_class_category': 'OMNY - Other'}


In [64]:
# Calculate the number of swipes by station complex
p3 = [
    # Group documents by station complex and calculate the sum of swipes for each group
    {"$group": {"_id": "$station_complex", "ridership": {"$sum": 1}}},
    # Project the _id field as station_complex and include the ridership count
    {"$project": {"station_complex": "$_id", "ridership": 1, "_id": 0}},
    # Sort ridership count in descending order
    {"$sort": {"ridership": -1}}
]

r3 = collection.aggregate(p3)

# Print the results
print("Number of Swipes by Station")
for result in r3:
    print(result)

Number of Swipes by Station
{'ridership': 31260, 'station_complex': 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'}
{'ridership': 30945, 'station_complex': '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)'}
{'ridership': 30700, 'station_complex': 'Grand Central-42 St (S,4,5,6,7)'}
{'ridership': 30630, 'station_complex': 'Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)'}
{'ridership': 30270, 'station_complex': '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'}
{'ridership': 30240, 'station_complex': '34 St-Penn Station (1,2,3)'}
{'ridership': 30205, 'station_complex': '14 St-Union Sq (L,N,Q,R,W,4,5,6)'}
{'ridership': 30090, 'station_complex': '14 St (F,M,1,2,3)/6 Av (L)'}
{'ridership': 29845, 'station_complex': '34 St-Penn Station (A,C,E)'}
{'ridership': 29695, 'station_complex': '59 St-Columbus Circle (A,B,C,D,1)'}
{'ridership': 29475, 'station_complex': '96 St (1,2,3)'}
{'ridership': 29455, 'station_complex': 'Fulton St (A,C,J,Z,2,3,4,5)'}
{'ridership': 29325, 'station_complex': '14 St (A,C,E)/

In [65]:
# Finding Coordinates which 1 shares 2&3 (World Trade Center - 96th)
# We search for stations that are 96th (1,2,3) and WTC Cortlandt
t1 = {"station_complex": {"$regex": "96 St \(1,2,3\)", "$options": "i"}}
a1 = {"station_complex": 1, "station_complex_id": 1,"latitude": 1,"longitude": 1, "_id": 0}
result_96 = collection.find_one(t1, a1)

# Query for WTC Cortlandt
t2 = {"station_complex": {"$regex": "WTC Cortlandt", "$options": "i"}}
result_wtc = collection.find_one(t2, a1)

# Print results which include coordinates and station id
print("96th:")
print(result_96)

print("World Trade Center:")
print(result_wtc)

96th:
{'station_complex_id': '310', 'station_complex': '96 St (1,2,3)', 'latitude': 40.7939186096, 'longitude': -73.9723205566}
World Trade Center:
{'station_complex_id': '328', 'station_complex': 'WTC Cortlandt (1)', 'latitude': 40.7118339539, 'longitude': -74.0121917725}


In [66]:
#Check to see which stations need express trains (1,2,3)
#find stations with the most swipes between WTC and 96th (1,2,3)
p6 = [
    # Filter documents by latitude within the range of WTC and 96th
    {"$match": {"latitude": {"$gte": 40.7118339539, "$lte": 40.7939186096}}},
    # Filter documents where 1 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(1"}},
        {"station_complex": {"$regex": ",1"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r6 = collection.aggregate(p6)

#Create a for loop to show the top 5 stations
count = 0

print("Stations With the most swipes between WTC and 96th (1,2,3)")
for result in r6:
    if count < 5:
        print(result)
        count += 1
    else:
        break

Stations With the most swipes between WTC and 96th (1,2,3)
{'ridership': 31260, 'station_complex': 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)', 'station_complex_id': '611'}
{'ridership': 30240, 'station_complex': '34 St-Penn Station (1,2,3)', 'station_complex_id': '318'}
{'ridership': 30090, 'station_complex': '14 St (F,M,1,2,3)/6 Av (L)', 'station_complex_id': '601'}
{'ridership': 29695, 'station_complex': '59 St-Columbus Circle (A,B,C,D,1)', 'station_complex_id': '614'}
{'ridership': 29475, 'station_complex': '96 St (1,2,3)', 'station_complex_id': '310'}


In [67]:
# Finding Coordinates for Uptown/Bronx 1 train (103rd - Van Cort 242nd)
# We search for stations that are 103rd (1) and 242nd (1)
t1 = {"station_complex": {"$regex": "Van Cort", "$options": "i"}}
a1 = {"station_complex": 1, "station_complex_id": 1,"latitude": 1,"longitude": 1, "_id": 0}
result_van = collection.find_one(t1, a1)

# Query for 103rd & 168th
t2 = {"station_complex": {"$regex": "103 St \(1\)", "$options": "i"}}
result_103 = collection.find_one(t2, a1)


# Print results for both stations
print("242nd:")
print(result_van)

print("103rd:")
print(result_103)


242nd:
{'station_complex_id': '293', 'station_complex': 'Van Cortlandt Park-242 St (1)', 'latitude': 40.8892478943, 'longitude': -73.8985824585}
103rd:
{'station_complex_id': '309', 'station_complex': '103 St (1)', 'latitude': 40.799446106, 'longitude': -73.9683761597}


In [68]:
# Find Uptown 1 train stations (M + Bx) with the most swipes
p7 = [
    # Filter documents by latitude within the range of Uptown stations (103rd - 242nd)
    {"$match": {"latitude": {"$gte": 40.799446106, "$lte": 40.8892478943}}},
    # Filter documents where 1 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(1"}},
        {"station_complex": {"$regex": ",1"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

# Execute the aggregation pipeline
r7 = collection.aggregate(p7)

#Create a for loop to show the top 5 stations
count = 0

# Display the results
print("1 Train Stations with the most Swipes Uptown (103rd - 242nd)")
for result in r7:
    if count < 5:
        print(result)
        count += 1
    else:
        break

1 Train Stations with the most Swipes Uptown (103rd - 242nd)
{'ridership': 28265, 'station_complex': '168 St (A,C,1)', 'station_complex_id': '605'}
{'ridership': 27530, 'station_complex': 'Cathedral Pkwy (110 St) (1)', 'station_complex_id': '308'}
{'ridership': 27315, 'station_complex': '103 St (1)', 'station_complex_id': '309'}
{'ridership': 27260, 'station_complex': '157 St (1)', 'station_complex_id': '303'}
{'ridership': 26795, 'station_complex': '181 St (1)', 'station_complex_id': '301'}


In [69]:
#Finding Coordinates of the 2 Train Uptown/Bronx
t1 = {"station_complex": {"$regex": "Central Park", "$options": "i"}}
a1 = {"station_complex": 1, "station_complex_id": 1,"latitude": 1,"longitude": 1, "_id": 0}
result_110 = collection.find_one(t1, a1)

# Query for Wakefield 241
t2 = {"station_complex": {"$regex": "Wakefield", "$options": "i"}}
result_wakefield = collection.find_one(t2, a1)


# Print results
print("110:")
print(result_110)

print("Wakefield:")
print(result_wakefield)

110:
{'station_complex_id': '441', 'station_complex': 'Central Park North (110 St) (2,3)', 'latitude': 40.7990760803, 'longitude': -73.9518203735}
Wakefield:
{'station_complex_id': '416', 'station_complex': 'Wakefield-241 St (2)', 'latitude': 40.9031257629, 'longitude': -73.8506164551}


In [70]:
#Which Uptown 2 trains need express
p8 = [
    # Filter documents by latitude within the range of Uptown/Bronx stations (103rd - 242nd)
    {"$match": {"latitude": {"$gte": 40.799076080322266, "$lte": 40.90312576293945}}},
    # Filter documents where 2 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(2"}},
        {"station_complex": {"$regex": ",2"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r8 = collection.aggregate(p8)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped in stations in Uptown/Bronx that have the 2 train")
for result in r8:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped in stations in Uptown/Bronx that have the 2 train
{'ridership': 27120, 'station_complex': '125 St (2,3)', 'station_complex_id': '439'}
{'ridership': 26700, 'station_complex': '135 St (2,3)', 'station_complex_id': '438'}
{'ridership': 26590, 'station_complex': '116 St (2,3)', 'station_complex_id': '440'}
{'ridership': 26515, 'station_complex': '149 St-Grand Concourse (2,4,5)', 'station_complex_id': '603'}
{'ridership': 26330, 'station_complex': 'Pelham Pkwy (2,5)', 'station_complex_id': '424'}


In [71]:
#Finding Which 2 Train Stations need express Brooklyn/Downtown
p9 = [
    # Filter documents by borough Brooklyn
    {"$match": {"borough": {"$regex": "Brooklyn"}}},
    # Filter documents where 2 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(2"}},
        {"station_complex": {"$regex": ",2"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r9 = collection.aggregate(p9)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Brooklyn that have the 2 train")
for result in r9:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Brooklyn that have the 2 train
{'ridership': 30630, 'station_complex': 'Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)', 'station_complex_id': '617'}
{'ridership': 27640, 'station_complex': 'Court St (R)/Borough Hall (2,3,4,5)', 'station_complex_id': '620'}
{'ridership': 27335, 'station_complex': 'Flatbush Av-Brooklyn College (2,5)', 'station_complex_id': '359'}
{'ridership': 26805, 'station_complex': 'Franklin Av (2,3,4,5)/Botanic Garden (S)', 'station_complex_id': '626'}
{'ridership': 25905, 'station_complex': 'Nevins St (2,3,4,5)', 'station_complex_id': '337'}


In [72]:
# Finding Which stations need express (4,5,6 train Manhattan)
p10 = [
    # Filter documents by borough Manhattan
    {"$match": {"borough": {"$regex": "Manhattan"}}},
    # Filter documents where 6 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(6"}},
        {"station_complex": {"$regex": ",6"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r10 = collection.aggregate(p10)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Manhattan on the Green Line (4,5,6)")
for result in r10:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Manhattan on the Green Line (4,5,6)
{'ridership': 30700, 'station_complex': 'Grand Central-42 St (S,4,5,6,7)', 'station_complex_id': '610'}
{'ridership': 30205, 'station_complex': '14 St-Union Sq (L,N,Q,R,W,4,5,6)', 'station_complex_id': '602'}
{'ridership': 29320, 'station_complex': 'Lexington Av (N,R,W)/59 St (4,5,6)', 'station_complex_id': '613'}
{'ridership': 28850, 'station_complex': '86 St (4,5,6)', 'station_complex_id': '397'}
{'ridership': 28645, 'station_complex': 'Lexington Av-53 St (E,M)/51 St (6)', 'station_complex_id': '612'}


In [73]:
#Now we check which stations need Express on 4 Train Uptown/Bronx
p11 = [
    # Filter documents by borough Bronx
    {"$match": {"borough": {"$regex": "Bronx"}}},
    # Filter documents where 4 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(4"}},
        {"station_complex": {"$regex": ",4"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r11 = collection.aggregate(p11)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Bronx that have the 4 train")
for result in r11:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Bronx that have the 4 train
{'ridership': 27635, 'station_complex': '161 St-Yankee Stadium (B,D,4)', 'station_complex_id': '604'}
{'ridership': 26960, 'station_complex': 'Fordham Rd (4)', 'station_complex_id': '382'}
{'ridership': 26515, 'station_complex': '149 St-Grand Concourse (2,4,5)', 'station_complex_id': '603'}
{'ridership': 26330, 'station_complex': 'Burnside Av (4)', 'station_complex_id': '384'}
{'ridership': 26115, 'station_complex': 'Kingsbridge Rd (4)', 'station_complex_id': '381'}


In [74]:
#Now we check which stations need Express on 6 Train Uptown/Bronx
p12 = [
    # Filter documents by borough Bronx
    {"$match": {"borough": {"$regex": "Bronx"}}},
    # Filter documents where 6 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(6"}},
        {"station_complex": {"$regex": ",6"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r12 = collection.aggregate(p12)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Bronx that have the 6 train")
for result in r12:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Bronx that have the 6 train
{'ridership': 27240, 'station_complex': 'Parkchester (6)', 'station_complex_id': '366'}
{'ridership': 26600, 'station_complex': 'Hunts Point Av (6)', 'station_complex_id': '371'}
{'ridership': 26600, 'station_complex': '3 Av-138 St (6)', 'station_complex_id': '377'}
{'ridership': 26205, 'station_complex': 'Pelham Bay Park (6)', 'station_complex_id': '360'}
{'ridership': 25300, 'station_complex': 'Morrison Av-Soundview (6)', 'station_complex_id': '368'}


In [75]:
#Which stations need Express on 7 Train Queens
p13 = [
    # Filter documents by borough Queens
    {"$match": {"borough": {"$regex": "Queens"}}},
    # Filter documents where 7 train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(7"}},
        {"station_complex": {"$regex": ",7"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r13 = collection.aggregate(p13)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Queens that have the 7 train")
for result in r13:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Queens that have the 7 train
{'ridership': 30945, 'station_complex': '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)', 'station_complex_id': '616'}
{'ridership': 27975, 'station_complex': 'Flushing-Main St (7)', 'station_complex_id': '447'}
{'ridership': 27355, 'station_complex': 'Court Sq (E,G,M,7)', 'station_complex_id': '606'}
{'ridership': 27270, 'station_complex': '82 St-Jackson Hts (7)', 'station_complex_id': '453'}
{'ridership': 27225, 'station_complex': 'Junction Blvd (7)', 'station_complex_id': '451'}


In [76]:
#Which stations need Express on F Train Brooklyn
p14 = [
    # Filter documents by borough Brooklyn
    {"$match": {"borough": {"$regex": "Brooklyn"}}},
    # Filter documents where F train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(F"}},
        {"station_complex": {"$regex": ",F"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r14 = collection.aggregate(p14)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Brooklyn that have the F train")
for result in r14:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Brooklyn that have the F train
{'ridership': 28580, 'station_complex': 'Jay St-MetroTech (A,C,F,R)', 'station_complex_id': '636'}
{'ridership': 27185, 'station_complex': 'Coney Island-Stillwell Av (D,F,N,Q)', 'station_complex_id': '58'}
{'ridership': 26850, 'station_complex': '4 Av (F,G)/9 St (R)', 'station_complex_id': '608'}
{'ridership': 26515, 'station_complex': 'Church Av (F,G)', 'station_complex_id': '243'}
{'ridership': 25680, 'station_complex': '7 Av (F,G)', 'station_complex_id': '240'}


In [77]:
#Which stations need Express on M Train Queens
p15 = [
    # Filter documents by borough Queens
    {"$match": {"borough": {"$regex": "Queens"}}},
    # Filter documents where M train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(M"}},
        {"station_complex": {"$regex": ",M"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r15 = collection.aggregate(p15)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Queens that have the M train")
for result in r15:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Queens that have the M train
{'ridership': 30945, 'station_complex': '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)', 'station_complex_id': '616'}
{'ridership': 28580, 'station_complex': 'Forest Hills-71 Av (E,F,M,R)', 'station_complex_id': '261'}
{'ridership': 27355, 'station_complex': 'Court Sq (E,G,M,7)', 'station_complex_id': '606'}
{'ridership': 26600, 'station_complex': '63 Dr-Rego Park (M,R)', 'station_complex_id': '263'}
{'ridership': 26510, 'station_complex': 'Woodhaven Blvd (M,R)', 'station_complex_id': '264'}


In [78]:
#Which stations need Express on A Train Queens
p16 = [
    # Filter documents by borough Queens
    {"$match": {"borough": {"$regex": "Queens"}}},
    # Filter documents where A train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(A"}},
        {"station_complex": {"$regex": ",A"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r16 = collection.aggregate(p16)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Queens that have the A train")
for result in r16:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Queens that have the A train
{'ridership': 25945, 'station_complex': 'Rockaway Blvd (A)', 'station_complex_id': '192'}
{'ridership': 25495, 'station_complex': 'Far Rockaway-Mott Av (A)', 'station_complex_id': '209'}
{'ridership': 25380, 'station_complex': 'Ozone Park-Lefferts Blvd (A)', 'station_complex_id': '195'}
{'ridership': 23915, 'station_complex': 'Howard Beach-JFK Airport (A)', 'station_complex_id': '198'}
{'ridership': 23875, 'station_complex': '80 St (A)', 'station_complex_id': '190'}


In [79]:
#Which stations need Express on N Train Queens
p17 = [
    # Filter documents by borough Queens
    {"$match": {"borough": {"$regex": "Queens"}}},
    # Filter documents where N train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(N"}},
        {"station_complex": {"$regex": ",N"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r17 = collection.aggregate(p17)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Queens that have the N train")
for result in r17:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Queens that have the N train
{'ridership': 26895, 'station_complex': 'Queensboro Plaza (7,N,W)', 'station_complex_id': '461'}
{'ridership': 26440, 'station_complex': 'Astoria-Ditmars Blvd (N,W)', 'station_complex_id': '1'}
{'ridership': 26405, 'station_complex': 'Broadway (N,W)', 'station_complex_id': '4'}
{'ridership': 26225, 'station_complex': '30 Av (N,W)', 'station_complex_id': '3'}
{'ridership': 25625, 'station_complex': 'Astoria Blvd (N,W)', 'station_complex_id': '2'}


In [80]:
#Which stations need Express on N Train Brooklyn
p18 = [
    # Filter documents by borough Brooklyn
    {"$match": {"borough": {"$regex": "Brooklyn"}}},
    # Filter documents where N train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(N"}},
        {"station_complex": {"$regex": ",N"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r18 = collection.aggregate(p18)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Brooklyn that have the N train")
for result in r18:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Brooklyn that have the N train
{'ridership': 30630, 'station_complex': 'Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)', 'station_complex_id': '617'}
{'ridership': 27185, 'station_complex': 'Coney Island-Stillwell Av (D,F,N,Q)', 'station_complex_id': '58'}
{'ridership': 26740, 'station_complex': '36 St (D,N,R)', 'station_complex_id': '32'}
{'ridership': 26245, 'station_complex': '59 St (N,R)', 'station_complex_id': '35'}
{'ridership': 25280, 'station_complex': '8 Av (N)', 'station_complex_id': '71'}


In [81]:
#Which stations need Express on Z Train Brooklyn & Queens
p19 = [
    # Filter documents by borough other than Manhattan
    {"$match": {"borough": {"$ne": "Manhattan"}}},
    # Filter documents where Z train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(Z"}},
        {"station_complex": {"$regex": ",Z"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r19 = collection.aggregate(p19)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations in Brooklyn that have the Z train")
for result in r19:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations in Brooklyn that have the Z train
{'ridership': 28125, 'station_complex': 'Sutphin Blvd-Archer Av-JFK Airport (E,J,Z)', 'station_complex_id': '279'}
{'ridership': 28020, 'station_complex': 'Jamaica Center-Parsons/Archer (E,J,Z)', 'station_complex_id': '278'}
{'ridership': 26615, 'station_complex': 'Marcy Av (M,J,Z)', 'station_complex_id': '101'}
{'ridership': 26545, 'station_complex': 'Broadway Junction (A,C,J,L,Z)', 'station_complex_id': '621'}
{'ridership': 26240, 'station_complex': 'Myrtle Av (M,J,Z)', 'station_complex_id': '97'}


In [82]:
#Which stations need Express on L Train
p20 = [
    # Filter documents by borough other than Manhattan
    {"$match": {"borough": {"$ne": "Manhattan"}}},
    # Filter documents where L train runs into the station complex
    {"$match": {"$or": [
        {"station_complex": {"$regex": "\(L"}},
        {"station_complex": {"$regex": ",L"}},
    ]}},
    # Group documents by station_complex and station_complex_id, calculate the sum of swipes for each group
    {"$group": {"_id": {"station_complex": "$station_complex", "station_complex_id": "$station_complex_id"}, "ridership": {"$sum": 1}}},
    # Project the station_complex, station_complex_id, and ridership fields
    {"$project": {"station_complex": "$_id.station_complex", "station_complex_id": "$_id.station_complex_id", "ridership": 1, "_id": 0}},
    # Sort ridership in descending order
    {"$sort": {"ridership": -1}}
]

r20 = collection.aggregate(p20)

# A counter for top 5 results
count = 0

# Display the results
print("The most swiped stations that have the L train")
for result in r20:
    if count < 5:
        print(result)
        count += 1
    else:
        break

The most swiped stations that have the L train
{'ridership': 27295, 'station_complex': 'Myrtle-Wyckoff Avs (L,M)', 'station_complex_id': '630'}
{'ridership': 26935, 'station_complex': 'Lorimer St (L)/Metropolitan Av (G)', 'station_complex_id': '629'}
{'ridership': 26545, 'station_complex': 'Broadway Junction (A,C,J,L,Z)', 'station_complex_id': '621'}
{'ridership': 25680, 'station_complex': 'Bedford Av (L)', 'station_complex_id': '120'}
{'ridership': 25140, 'station_complex': 'New Lots Av (L)', 'station_complex_id': '136'}


In [85]:
import numpy as np
import pandas as pd
import json
import os as os

In [86]:
#Write the output to a webpage using Flask
from flask import Flask, render_template
app = Flask("newApp")

In [87]:
@app.route('/', methods=['GET', 'POST'])
def index():
    # Initialize variables
    borough_results = []
    line_results = []
    line = None  # Set a default value for 'line'

    if request.method == 'POST':
        selected_boroughs = request.form.getlist('borough')  # Retrieves all checked boroughs
        line = request.form.get('line')  # Retrieve the line from the form input
        if line:  # Check if a line was entered
            regex = r"\(" + line + "|" + "," + line
            line_pipeline = [
                {"$match": {"$and": [
                    {"station_complex": {"$regex": regex}},
                    {"borough": {"$in": selected_boroughs}}
                ]}},
                {"$group": {"_id": {"station_complex": "$station_complex", "borough": "$borough"}, "ridership": {"$sum": 1}}},
                {"$project": {"station_complex": "$_id.station_complex", "borough": "$_id.borough", "ridership": 1, "_id": 0}},
                {"$sort": {"ridership": -1}},
                {"$limit": 5}
            ]
            line_results = list(collection.aggregate(line_pipeline))
    else:
        # If it's a GET request, display the default borough results
        borough_pipeline = [
            {"$group": {"_id": "$borough", "ridership": {"$sum": 1}}},
            {"$project": {"borough": "$_id", "ridership": 1, "_id": 0}},
            {"$sort": {"ridership": -1}}
        ]
        borough_results = list(collection.aggregate(borough_pipeline))

    return render_template('demo.html', borough_results=borough_results, line_results=line_results, line=line)

In [88]:
#Starts the Flask web server on the local machine at port 5001.
app.run(host='localhost', port=5001)

 * Serving Flask app 'newApp'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Apr/2024 18:41:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2024 18:42:16] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2024 18:42:37] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2024 18:42:46] "POST / HTTP/1.1" 200 -
